# Problem Statement

Using the Craigslist Vehicles Dataset available on Kaggle (https://www.kaggle.com/datasets/mbaabuharun/craigslist-vehicles), we'd like you to create a Time-Series Model following the approach outlined below.

Here are the key steps:

1. Start by addressing missing values in the dataset. You can handle this by filling in missing values with the median for numerical columns and the mode for categorical columns. 
2. Ensure that the data types of the columns are appropriate. Specifically, make sure to convert the 'posting_date' column to a datetime data type.
3. Utilize the 'posting_date' column to create a datetime index for the dataset. This will facilitate the analysis of temporal patterns.
4. With clean data, explore it using various visualizations and statistical analysis techniques. This step is crucial for understanding temporal patterns, identifying seasonal trends, and analyzing demand-supply dynamics by region and vehicle type.
5. Build the time-series chart.
6. Finally, create a GitHub Repository and push your work there, also document your process through each of the steps and demonstrate your understanding by implementing them on the dataset.


In [30]:
# Importing libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.impute import SimpleImputer

%matplotlib inline
pd.set_option("display.max_columns", None)

In [31]:
def data_outlook(data: pd.DataFrame()):
    """
    Input:
    A dataframe

    Returns:
    A general overview of the data and its characteristics
    """
    # Shape
    shape = data.shape
    print(f"There are {shape[0]} records and {shape[1]} features in the data", end='\n\n')
    # Data Info
    print(data.info())
    # Numerical data description
    display(data.describe())
    # 

In [32]:
# Load data
data = pd.read_csv(r"craigslist_vehicles.csv")

data.head(3)

,Unnamed: 0,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date,removal_date
0,362773,7307679724,https://abilene.craigslist.org/ctd/d/abilene-2...,abilene,https://abilene.craigslist.org,4500,2002.0,bmw,x5,NaN,NaN,gas,184000.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,https://images.craigslist.org/00m0m_iba78h8ty9...,"$4,500 Cash 2002 BMW X5 8 cylinder 4.4L moto...",NaN,tx,32.401556,-99.884713,2021-04-16 00:00:00+00:00,2021-05-02 00:00:00+00:00
1,362712,7311833696,https://abilene.craigslist.org/ctd/d/abilene-2...,abilene,https://abilene.craigslist.org,4500,2002.0,bmw,x5,NaN,NaN,gas,184000.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,https://images.craigslist.org/00m0m_iba78h8ty9...,"$4,500 Cash 2002 BMW X5 8 cylinder 4.4L moto...",NaN,tx,32.401556,-99.884713,2021-04-24 00:00:00+00:00,2021-04-28 00:00:00+00:00
2,362722,7311441996,https://abilene.craigslist.org/ctd/d/abilene-2...,abilene,https://abilene.craigslist.org,4900,2006.0,toyota,camry,excellent,4 cylinders,gas,184930.0,clean,automatic,NaN,fwd,NaN,sedan,silver,https://images.craigslist.org/00808_5FkOw2aGjA...,2006 TOYOTA CAMRY LE Sedan Ready To Upgrade ...,NaN,tx,32.453848,-99.787900,2021-04-23 00:00:00+00:00,2021-05-25 00:00:00+00:00


In [33]:
# Data overview
data_outlook(data)

There are 426880 records and 28 features in the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    426880 non-null  int64  
 1   id            426880 non-null  int64  
 2   url           426880 non-null  object 
 3   region        426880 non-null  object 
 4   region_url    426880 non-null  object 
 5   price         426880 non-null  int64  
 6   year          425675 non-null  float64
 7   manufacturer  409234 non-null  object 
 8   model         421603 non-null  object 
 9   condition     252776 non-null  object 
 10  cylinders     249202 non-null  object 
 11  fuel          423867 non-null  object 
 12  odometer      422480 non-null  float64
 13  title_status  418638 non-null  object 
 14  transmission  424324 non-null  object 
 15  VIN           265838 non-null  object 
 16  drive         296313 non-null  object 

,Unnamed: 0,id,price,year,odometer,county,lat,long
count,426880.000000,4.268800e+05,4.268800e+05,425675.000000,4.224800e+05,0.0,420331.000000,420331.000000
mean,213439.500000,7.311487e+09,7.519903e+04,2011.235191,9.804333e+04,NaN,38.493940,-94.748599
std,123229.785793,4.473170e+06,1.218228e+07,9.452120,2.138815e+05,NaN,5.841533,18.365462
min,0.000000,7.207408e+09,0.000000e+00,1900.000000,0.000000e+00,NaN,-84.122245,-159.827728
25%,106719.750000,7.308143e+09,5.900000e+03,2008.000000,3.770400e+04,NaN,34.601900,-111.939847
50%,213439.500000,7.312621e+09,1.395000e+04,2013.000000,8.554800e+04,NaN,39.150100,-88.432600
75%,320159.250000,7.315254e+09,2.648575e+04,2017.000000,1.335425e+05,NaN,42.398900,-80.832039
max,426879.000000,7.317101e+09,3.736929e+09,2022.000000,1.000000e+07,NaN,82.390818,173.885502


In [34]:
# Check missing values
round(data.isna().sum() / data.shape[0] * 100, 2)

Unnamed: 0        0.00
id                0.00
url               0.00
region            0.00
region_url        0.00
price             0.00
year              0.28
manufacturer      4.13
model             1.24
condition        40.79
cylinders        41.62
fuel              0.71
odometer          1.03
title_status      1.93
transmission      0.60
VIN              37.73
drive            30.59
size             71.77
type             21.75
paint_color      30.50
image_url         0.02
description       0.02
county          100.00
state             0.00
lat               1.53
long              1.53
posting_date      0.02
removal_date      0.02
dtype: float64

Observations and conclusions made: 

- There are a lot of missing values in the data. 
- There are some unnecessary features which I will drop.
- Some features have to be renamed 

In [35]:
# Dropping unnecessary features
data.drop(columns=['county', 'VIN', 'size', 'url', 'region_url', 'image_url', 'Unnamed: 0', 'description', 'lat', 'long'], inplace=True) # some are due to the large number of missing values

# Renaming some features
data.rename({'odometer': 'milage', 'year': 'manufacture_year'}, axis=1, inplace=True)

Missing values can be filled by text processing the description feature, however, due to time, I will explore that line some other time. 

I will fill in the missing values with median for numerical features and mode for categorical features.

In [36]:
# Dropping nan records for posting date since it will be used as an index later.
data.dropna(subset=['posting_date'], inplace=True)

In [37]:
# Filling missing values
numerical_df = data.select_dtypes(include=['int', 'float'])
non_numeric_df = data.select_dtypes(exclude=['int', 'float'])

# Instantiating the missing value imputers
numerical_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
non_numeric_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

# Fitting and transforming
numerical_df = pd.DataFrame(numerical_imputer.fit_transform(numerical_df), columns=numerical_df.columns, index=numerical_df.index)
non_numeric_df = pd.DataFrame(non_numeric_imputer.fit_transform(non_numeric_df), columns=non_numeric_df.columns, index=non_numeric_df.index)

# Forming back the whole dataframe
data = numerical_df.join(non_numeric_df)

**Correcting the datatypes of the features in the data**

In [39]:
# Changing data type for temporal data
data['posting_date'] = pd.to_datetime(data['posting_date'])
data['removal_date'] = pd.to_datetime(data['removal_date'])

In [40]:
# Posting date description
data['posting_date'].describe()

count                                 426812
mean     2021-04-23 08:50:52.930095872+00:00
min                2021-04-04 00:00:00+00:00
25%                2021-04-17 00:00:00+00:00
50%                2021-04-26 00:00:00+00:00
75%                2021-05-01 00:00:00+00:00
max                2021-05-05 00:00:00+00:00
Name: posting_date, dtype: object

The data spans the period of April 4th to May 5th 2021. That is approximately a month. 

Since I want to drop the removal date feature, I want to engineer a feature called days_in_store that is a the days that it has taken for a vehicle to be sold/removed from the website.

        days_in_store = removal date - posting date
        

In [41]:
data['days_in_store'] = (data['removal_date'] - data['posting_date']).dt.days

# Dropping removal date feature
data.drop(columns='removal_date', inplace=True)

In [42]:
# Correcting other datatypes
data[['region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'drive', 'type', 'state']] = data[['region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'drive', 'type', 'state']].astype(dtype='category')

data['id'] = data[['id']].astype(dtype='object')

In [43]:
# Making posting_date the index column to change data time series
data.set_index(keys='posting_date', drop=True, inplace=True)

In [44]:
data.head()

,id,price,manufacture_year,milage,region,manufacturer,model,condition,cylinders,fuel,title_status,transmission,drive,type,paint_color,state,days_in_store
posting_date,,,,,,,,,,,,,,,,,
2021-04-16 00:00:00+00:00,7307679724.0,4500.0,2002.0,184000.0,abilene,bmw,x5,good,6 cylinders,gas,clean,automatic,4wd,sedan,white,tx,16
2021-04-24 00:00:00+00:00,7311833696.0,4500.0,2002.0,184000.0,abilene,bmw,x5,good,6 cylinders,gas,clean,automatic,4wd,sedan,white,tx,4
2021-04-23 00:00:00+00:00,7311441996.0,4900.0,2006.0,184930.0,abilene,toyota,camry,excellent,4 cylinders,gas,clean,automatic,fwd,sedan,silver,tx,32
2021-04-16 00:00:00+00:00,7307680715.0,6500.0,2008.0,206000.0,abilene,ford,expedition,good,6 cylinders,gas,clean,automatic,4wd,sedan,white,tx,10
2021-04-24 00:00:00+00:00,7311834578.0,6500.0,2008.0,206000.0,abilene,ford,expedition,good,6 cylinders,gas,clean,automatic,4wd,sedan,white,tx,18


The data is now clean I can proceed to its analysis and other processes.